In [5]:
import urllib.parse as urlparse
import pandas as pd
import requests as rq

In [31]:
__API_KEY__ = "O81bHEfH6GzFRVgj6CTXyPGTYAsqNEVJ"
departure_point = 'Los Angeles'
delivery_point = 'Irvine'
departure_time = '2021-10-02T08:00:00'

In [27]:
def search_city_coords(city, key):
    coordinates_base_url = "https://api.tomtom.com/search/2/search/"+urlparse.quote(city)+".json?minFuzzyLevel=1&maxFuzzyLevel=2&view=Unified&relatedPois=off&key="+key
    json_response = rq.get(coordinates_base_url).json()
    latitude = json_response['results'][0]['position']['lat']
    longitude = json_response['results'][0]['position']['lon']
    position = str(latitude)+","+str(longitude)
    return position

In [104]:
def get_routes_info(start, stop, departAt, key):
    base_url = "https://api.tomtom.com/routing/1/calculateRoute/"+start+":"+stop+"/json?maxAlternatives=5&instructionsType=text&departAt="+departure_time+"&key="+key
    json_response = rq.get(base_url).json()
    routes_obj = json_response['routes']
    obj_length = len(routes_obj)
    for i in range(obj_length):
        if (i == 0):
            summary_df = pd.DataFrame([routes_obj[0]['summary']])
        else:
            summary_df = summary_df.append((routes_obj[i]['summary']),ignore_index=True)
    return summary_df

In [105]:
departure_point_coords = search_city_coords(departure_point, __API_KEY__)
delivery_point_coords = search_city_coords(delivery_point, __API_KEY__)

summary_df = get_routes_info(departure_point_coords, delivery_point_coords, departure_time, __API_KEY__)

In [106]:
summary_df.iloc[: , :6]

,lengthInMeters,travelTimeInSeconds,trafficDelayInSeconds,trafficLengthInMeters,departureTime,arrivalTime
0,64191,2583,0,0,2021-10-02T08:00:00-07:00,2021-10-02T08:43:02-07:00
1,75233,2861,0,0,2021-10-02T08:00:00-07:00,2021-10-02T08:47:40-07:00
2,68624,2696,0,0,2021-10-02T08:00:00-07:00,2021-10-02T08:44:55-07:00
3,84159,3154,0,0,2021-10-02T08:00:00-07:00,2021-10-02T08:52:33-07:00
4,73332,2854,0,0,2021-10-02T08:00:00-07:00,2021-10-02T08:47:33-07:00
5,83048,3142,0,0,2021-10-02T08:00:00-07:00,2021-10-02T08:52:21-07:00


In [107]:
min_travel_time = summary_df['travelTimeInSeconds'].min()
print("Minimum travel time is: ",min_travel_time," seconds")

Minimum travel time is:  2583  seconds


In [108]:
min_pos = summary_df['travelTimeInSeconds'].idxmin()
min_pos

0

In [119]:
def get_guidance(start, stop, departAt, position, key):
    guide = ''
    base_url = "https://api.tomtom.com/routing/1/calculateRoute/"+start+":"+stop+"/json?maxAlternatives=5&instructionsType=text&departAt="+departure_time+"&key="+key
    json_response = rq.get(base_url).json()
    guidance_obj = json_response['routes'][position]['guidance']['instructionGroups']
    obj_length = len(guidance_obj)
    for i in range(obj_length):
        if i==0:
            guide = guidance_obj[0]['groupMessage']
        else:
            guide = guide+". "+ guidance_obj[i]['groupMessage']
    return guide

In [128]:
shortest_route = get_guidance(departure_point_coords, delivery_point_coords, departure_time,min_pos, __API_KEY__)
print("HELLO THERE, FOLLOW THIS ROUTE FOR A FASTER DELIVERY: ", shortest_route,"\n\nThank you and have a safe ride")

HELLO THERE, FOLLOW THIS ROUTE FOR A FASTER DELIVERY: 

 Leave from N Main St. Take the United States Highway 101 S/US-101 S, Take the Interstate Highway 5 S/I-5 S toward Santa Ana, Take the Jamboree Rd, Jamboree Rd S, Barranca Pkwy. Continue to your destination at Harvard Ave 

Thank you and have a safe ride
